In [1]:
library(tidyverse)
library(magrittr) # better handling of pipes
library(purrr) # to work with lists and map functions
library(glue) # to paste strings
library(stringr) # to hand strings
library(rvest) # rvest makes scraping easier
library(polite)

-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.2.0     v purrr   0.3.2
v tibble  2.1.3     v dplyr   0.8.3
v tidyr   0.8.3     v stringr 1.4.0
v readr   1.3.1     v forcats 0.4.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Attaching package: 'magrittr'

The following object is masked from 'package:purrr':

    set_names

The following object is masked from 'package:tidyr':

    extract


Attaching package: 'glue'

The following object is masked from 'package:dplyr':

    collapse

Loading required package: xml2

Attaching package: 'rvest'

The following object is masked from 'package:purrr':

    pluck

The following object is masked from 'package:readr':

    guess_encoding



In [47]:
years <- seq(1950, 2019, 1) #first number = first year for scrape, second number = last year for scraping

In [3]:

#Grabs each movie link for provided years
data <- function(years){
    movie_links <- NULL
    movie_links <- list()
    for(year in years){
        url_titles <- glue("https://www.rottentomatoes.com/top/bestofrt/?year={year}")
        base_page <- read_html(url_titles)

        movie_links <- append(movie_links, list(base_page %>%
            html_nodes(".allow-overflow") %>%
            html_nodes(".articleLink") %>%
            html_attr("href")
            ))

            }
    return(movie_links)
    }



In [51]:
movie_links <- data(years)
movie_links

[[1]]
  [1] "/m/black_panther_2018"                   
  [2] "/m/mission_impossible_fallout"           
  [3] "/m/blackkklansman"                       
  [4] "/m/spider_man_into_the_spider_verse"     
  [5] "/m/roma_2018"                            
  [6] "/m/a_star_is_born_2018"                  
  [7] "/m/eighth_grade"                         
  [8] "/m/a_quiet_place_2018"                   
  [9] "/m/can_you_ever_forgive_me"              
 [10] "/m/paddington_2"                         
 [11] "/m/the_favourite_2018"                   
 [12] "/m/incredibles_2"                        
 [13] "/m/call_me_by_your_name"                 
 [14] "/m/leave_no_trace"                       
 [15] "/m/widows_2018"                          
 [16] "/m/shoplifters"                          
 [17] "/m/wont_you_be_my_neighbor"              
 [18] "/m/the_death_of_stalin"                  
 [19] "/m/the_hate_u_give"                      
 [20] "/m/ant_man_and_the_wasp"                 
 [21] "/m/first_man"                            
 [22] "/m/crazy_rich_asians"                    
 [23] "/m/sorry_to_bother_you_2018"             
 [24] "/m/phantom_thread"                       
 [25] "/m/i_tonya"                              
 [26] "/m/avengers_infinity_war"                
 [27] "/m/the_post"                             
 [28] "/m/isle_of_dogs_2018"                    
 [29] "/m/hereditary"                           
 [30] "/m/the_old_man_and_the_gun"              
 [31] "/m/the_rider"                            
 [32] "/m/minding_the_gap"                      
 [33] "/m/mcqueen"                              
 [34] "/m/summer_1993"                          
 [35] "/m/three_identical_strangers"            
 [36] "/m/first_reformed"                       
 [37] "/m/free_solo"                            
 [38] "/m/burning_2018"                         
 [39] "/m/searching_2018"                       
 [40] "/m/the_guilty_2018"                      
 [41] "/m/wildlife_2018"                        
 [42] "/m/a_fantastic_woman"                    
 [43] "/m/bumblebee"                            
 [44] "/m/tea_with_the_dames"                   
 [45] "/m/shirkers"                             
 [46] "/m/cold_war_2018"                        
 [47] "/m/rbg"                                  
 [48] "/m/the_tale"                             
 [49] "/m/border_2018"                          
 [50] "/m/love_simon"                           
 [51] "/m/oh_lucy_2018"                         
 [52] "/m/loveless_2018"                        
 [53] "/m/night_comes_on"                       
 [54] "/m/hannah_gadsby_nanette"                
 [55] "/m/mandy_2018"                           
 [56] "/m/blindspotting"                        
 [57] "/m/deadpool_2"                           
 [58] "/m/you_were_never_really_here"           
 [59] "/m/ralph_breaks_the_internet"            
 [60] "/m/the_ballad_of_buster_scruggs"         
 [61] "/m/zama"                                 
 [62] "/m/in_between_2018"                      
 [63] "/m/annihilation"                         
 [64] "/m/thunder_road_2018"                    
 [65] "/m/the_cakemaker"                        
 [66] "/m/science_fair"                         
 [67] "/m/i_am_not_a_witch"                     
 [68] "/m/foxtrot_2018"                         
 [69] "/m/custody_2018"                         
 [70] "/m/sweet_country_2018"                   
 [71] "/m/hale_county_this_morning_this_evening"
 [72] "/m/ryuichi_sakamoto_coda"                
 [73] "/m/to_all_the_boys_ive_loved_before"     
 [74] "/m/marlina_the_murderer_in_four_acts"    
 [75] "/m/rodents_of_unusual_size"              
 [76] "/m/leaning_into_the_wind"                
 [77] "/m/pick_of_the_litter"                   
 [78] "/m/lean_on_pete"                         
 [79] "/m/western_2018"                         
 [80] "/m/blaze_2018"                           
 [81] "/m/revenge_2018"                         
 [82] "/m/the_endless"   

In [67]:
get_info <- function(link){
    
    #Retrieve critic and audience rating
    movie_base_page <- glue("https://www.rottentomatoes.com{link}")
    web_data <- movie_base_page %>%
        read_html()
    movie_reviews <- web_data %>%
        html_nodes(".js-scoreboard-container") %>%
        html_nodes(".mop-ratings-wrap__percentage") %>%
        html_text() %>% 
        trimws() 
    #Retrieves the movie title
    movie_title <- web_data %>%
        html_nodes(".mop-ratings-wrap__title--top") %>%
        html_text()
    
    #grab supplementary data
    movie_info_labels <- web_data %>%
        html_nodes('.content-meta.info') %>%
        html_nodes(".meta-row.clearfix")%>%
        html_nodes('.meta-label')%>%
        html_text() %>%
        trimws()
    movie_info_data <- web_data %>%
        html_nodes('.content-meta.info') %>%
        html_nodes(".meta-row.clearfix")%>%
        html_nodes('.meta-value')%>%
        html_text() %>%
        trimws() 
    
    #formats supplementary data into key-value pairs for processing
    movie_info <- tibble(movie_info_labels, movie_info_data)
    movie_info <- spread(movie_info, 
                        key = movie_info_labels,
                        value = movie_info_data)
    

    
    #Retrieves value if the value is there otherwise sets value to NA
    if ("Rating:" %in% colnames(movie_info)){
        rating <- movie_info$"Rating:"
    } else {
        rating <- NA
    }
    if ("Runtime:" %in% colnames(movie_info)){
        runtime <- movie_info$"Runtime:"
        runtime <- gsub("[^0-9\\.]", "", runtime) %>%
                    as.numeric()
    } else {
        runtime <- NA
    }
    if ("Studio:" %in% colnames(movie_info)){
        studio <- movie_info$"Studio:"
    } else {
        studio <- NA
    }
    
    if ("Genre:" %in% colnames(movie_info)){
        genre <- movie_info$"Genre:" %>% 
                str_replace_all(fixed(" "), "") %>% 
                str_replace_all(fixed("\n"), "") %>%
                strsplit(",")
    } else {
        genre <- NA
    }
  
    if ("Box Office:" %in% colnames(movie_info)){
        box_office <- movie_info$"Box Office:" 
        box_office <- gsub("[^0-9\\.]", "", box_office) %>%
        as.numeric()
        
    } else {
        box_office <- NA
        
    }
    
    #place data into a tibble
    processed_data <- tibble("Movie Title" = movie_title, 
                  "Critic Score (%)" = readr::parse_number(movie_reviews[1]), 
                  "Audience Score (%)" = readr::parse_number(movie_reviews[2]),
                  "Movie Rating" = rating,
                  "Box Office ($)" = as.numeric(box_office), 
                  "Runtime (m)" = runtime,
                  "Genre" = genre,
                  "Studio" = studio)

    return(processed_data)
}

# Base Test 
# get_info(movie_links[[1]][[1]])


# puts all tibbles together into one 
all_df <- tibble()
    for (decade in movie_links){
      for (link in decade){
        temp <- get_info(link)
        all_df <- rbind(all_df, temp)    #combines each movie tibble retrieved into a single tibble
    }
}



Movie Title,Critic Score (%),Audience Score (%),Movie Rating,Box Office ($),Runtime (m),Genre,Studio
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<list>,<chr>
Black Panther,96,79,"PG-13 (for prolonged sequences of action violence, and a brief rude gesture)",501105037,135,"Action&Adventure , Drama , ScienceFiction&Fantasy",Marvel Studios


In [68]:
all_df

Movie Title,Critic Score (%),Audience Score (%),Movie Rating,Box Office ($),Runtime (m),Genre,Studio
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<list>,<chr>
Black Panther,96,79,"PG-13 (for prolonged sequences of action violence, and a brief rude gesture)",501105037,135,"Action&Adventure , Drama , ScienceFiction&Fantasy",Marvel Studios
Mission: Impossible - Fallout,97,88,"PG-13 (for violence and intense sequences of action, and for brief strong language)",NA,147,"Action&Adventure, Drama , Mystery&Suspense",Paramount Pictures
BlacKkKlansman,96,82,"R (for language throughout, including racial epithets, and for disturbing/violent material and some sexual references)",NA,135,"Comedy, Drama",Focus Features
Spider-Man: Into the Spider-Verse,97,93,"PG (for frenetic sequences of animated action violence, thematic elements, and mild language)",NA,100,"Action&Adventure , Animation , Kids&Family , ScienceFiction&Fantasy",Sony Pictures
Roma,96,71,"R (for graphic nudity, some disturbing images, and language)",NA,135,Drama,Netflix
A Star Is Born,90,79,"R (for language throughout, some sexuality/nudity and substance abuse)",NA,135,Drama,Warner Bros. Pictures
Eighth Grade,99,83,R (for language and some sexual material),NA,94,Comedy,A24
A Quiet Place,95,83,PG-13 (for terror and some bloody images),NA,90,"Drama , Horror , Mystery&Suspense",Paramount Pictures
Can You Ever Forgive Me?,98,81,"R (for language including some sexual references, and brief drug use)",NA,107,"Comedy, Drama",Fox Searchlight Pictures
